In [1]:
# ENV["PYTHON"] = "/home/users/kww231/virtualenvs/phd/bin/python"
# rm(Pkg.dir("PyCall","deps","PYTHON")); Pkg.checkout("PyCall"); Pkg.build("PyCall")

include("../src/obs_funcs.jl")
include("../src/mod_funcs.jl")

massbincounts

In [2]:
# here are the HOD parameters to be used for each of the samples

# for the -18 magnitude threshold, parameters are taken from Table 3 in Zehavi et al. 2011
m_th_18 = -18.0
paramdict_18 = Dict("alpha" => 1.04,
                    "logM0" =>  9.81,
                    "logM1" => 12.42,
                    "logMmin" => 11.18,
                    "sigma_logM" => 0.19)

# for the -19 magnitude threshold, parameters are taken from Sinha et al. 2016 best-fit values
# from table 3, using the values for Planck Cosmology with Mvir (as Chinchilla halos are defined) and no PCA uncertainties
m_th_19 = -19.0
paramdict_19 = Dict("alpha" => 0.95,       #  1.02 in Zehavi    1.03 in Zentner
                    "logM0" => 11.14,      #  9.77
                    "logM1" => 12.80,      # 12.63             12.78 whether I use Zehavi M0 or not
                    "logMmin" => 11.58,    # 11.45             11.72
                    "sigma_logM" => 0.05)  #  0.19              0.70

# for the -20 magnitude threshold, parameters are taken from Table 2 of Zentner et al. 2016, using M_0 from Zehavi
m_th_20 = -20.0
paramdict_20 = Dict("alpha" => 1.16,       #  1.0 in Zehavi
                    "logM0" =>  12.35,
                    "logM1" => 13.23,      # 12.98
                    "logMmin" => 11.96,    # 11.83
                    "sigma_logM" => 0.38)  #  0.25

m_th_arr = [m_th_18, m_th_19, m_th_20]
paramdicts = [paramdict_18, paramdict_19, paramdict_20]

3-element Array{Dict{String,Float64},1}:
 Dict("sigma_logM"=>0.19,"logMmin"=>11.18,"alpha"=>1.04,"logM1"=>12.42,"logM0"=>9.81) 
 Dict("sigma_logM"=>0.05,"logMmin"=>11.58,"alpha"=>0.95,"logM1"=>12.8,"logM0"=>11.14) 
 Dict("sigma_logM"=>0.38,"logMmin"=>11.96,"alpha"=>1.16,"logM1"=>13.23,"logM0"=>12.35)

In [27]:
# set which sample to generate a mock for and grab the data
sample_num = 3
datdf, vols = load_groupdat(sample_num, quenform = "both")

(47784×46 DataFrames.DataFrame. Omitted printing of 39 columns
│ Row   │ foo    │ galid  │ groupid │ cenid  │ rband    │ Psat │ Mh         │
├───────┼────────┼────────┼─────────┼────────┼──────────┼──────┼────────────┤
│ 1     │ PROB10 │ 3      │ 26747   │ 3      │ -20.9869 │ 0.0  │ 3.03811e12 │
│ 2     │ PROB10 │ 17     │ 46451   │ 17     │ -20.4593 │ 0.0  │ 1.67686e12 │
│ 3     │ PROB10 │ 21     │ 29108   │ 21     │ -21.3315 │ 0.0  │ 2.77697e12 │
│ 4     │ PROB10 │ 53     │ 15187   │ 53     │ -21.1077 │ 0.0  │ 5.48889e12 │
│ 5     │ PROB10 │ 81     │ 36849   │ 81     │ -20.5768 │ 0.0  │ 2.15532e12 │
│ 6     │ PROB10 │ 83     │ 24536   │ 83     │ -20.8211 │ 0.0  │ 3.32973e12 │
│ 7     │ PROB10 │ 85     │ 52256   │ 85     │ -20.6159 │ 0.0  │ 1.47419e12 │
│ 8     │ PROB10 │ 90     │ 25369   │ 90     │ -20.8662 │ 0.0  │ 3.21402e12 │
│ 9     │ PROB10 │ 93     │ 10451   │ 93     │ -21.3781 │ 0.0  │ 7.98381e12 │
│ 10    │ PROB10 │ 112    │ 1095    │ 112    │ -21.8533 │ 0.0  │ 4.79077e13 │
│

In [28]:
# get another dataframe of just the quenched galaxies and their masses for counting quenched fraction
quendf = datdf[datdf[:Dn4000] .> 1.6, [:log10M]]

,log10M
1,10.512
2,10.3666
3,10.6508
4,10.4301
5,10.5335
6,10.5255
7,10.7409
8,10.9336
9,10.452
10,10.3502


In [29]:
# generate the mock galaxies
mockgals = get_mock(paramdicts[sample_num], m_th_arr[sample_num], 3)

383665×6 Array{Float64,2}:
 1.62049e9   23.8908   178.352   69.3618  1.0   4.0
 1.62049e9   54.2385   147.212   86.2612  1.0   4.0
 1.62057e9   17.5035   142.147  121.991   1.0   4.0
 1.62057e9   30.9299   185.085  119.293   1.0   4.0
 1.62057e9    3.68962  147.032  121.787   1.0   4.0
 1.61988e9   46.9333   134.869   85.5845  1.0   4.0
 1.62057e9   10.0025   186.166  128.661   1.0   4.0
 1.62111e9   39.2183   196.085   89.3759  1.0   4.0
 1.62049e9   38.9772   181.184   67.3124  1.0   4.0
 1.62049e9   28.9453   165.548   72.4649  1.0   4.0
 1.6205e9    55.9567   140.022   83.2813  1.0   4.0
 1.62057e9   62.4508   162.055  106.218   1.0   4.0
 1.62057e9   20.3627   143.978  122.714   1.0   4.0
 ⋮                                              ⋮  
 1.63305e9  252.357    357.58   257.892   0.0  17.0
 1.63305e9  224.616    362.029  263.523   0.0  17.0
 1.63237e9  237.126    349.875  237.058   0.0  17.0
 1.63298e9  238.778    398.423  221.981   0.0  17.0
 1.63298e9  238.291    374.415  255.4

In [30]:
# compute the environment densities of the galaxies in the mock
deltas = get_rho(mockgals[:, 2:4], 400.0, 10.0)

383665-element Array{Float64,1}:
 2.66376 
 2.82305 
 0.274351
 0.194704
 0.433645
 1.86729 
 0.274351
 1.46906 
 0.274351
 0.911527
 2.34517 
 0.632763
 0.393822
 ⋮       
 4.2567  
 0.513292
 0.154881
 1.46906 
 2.74341 
 1.708   
 1.78764 
 3.06199 
 2.66376 
 2.62394 
 3.14164 
 2.10623 

In [7]:
# grab the halos to get stellar masses
halodf = load_halodat()

,halo_id,halo_upid,x,y,z,vx,vy,vz,hfoo1,hfoo2,halo_mvir,halo_mz_05,halo_mz_1,halo_mz_2,halo_mz_4,halo_mz_8,halo_ac,halo_ac_mpeak,halo_nfw_conc,halo_spin,hfoo3,halo_rvir,logMh,Mpeak,logMpeak,Mbar
1,1620490488,-1,23.8908,178.352,71.8972,254.19,-195.17,-253.54,758.46,780.72,1.303e14,1.117e14,1.016e14,9.289e13,6.113e13,4.885e13,0.51462,0.51462,4.34886,0.03889,1620490488,1.25141,14.1149,1.303e14,14.1149,2.03268e13
2,1620489517,-1,54.2385,147.212,83.8446,132.18,-243.42,241.66,746.88,787.27,1.031e14,1.009e14,9.834e13,8.531e13,6.773e13,5.168e13,0.602988,0.602988,6.31183,0.03163,1620489517,1.15745,14.0133,1.031e14,14.0133,1.60836e13
3,1620568787,-1,17.5035,142.147,121.233,206.25,52.6,75.76,800.83,833.51,1.023e14,9.305e13,8.565e13,8.209e13,7.753e13,5.354e13,0.465599,0.465599,8.92032,0.01292,1620568787,1.15445,14.0099,1.023e14,14.0099,1.59588e13
4,1620569234,-1,30.9299,185.085,116.149,100.4,211.18,314.45,759.85,808.61,8.734e13,8.65e13,8.481e13,7.882e13,6.102e13,4.87e13,0.474902,0.474902,8.7816,0.02291,1620569234,1.09519,13.9412,8.734e13,13.9412,1.3625e13
5,1620569405,-1,3.68962,147.032,119.642,318.78,-36.68,214.57,655.71,762.05,7.256e13,7.268e13,7.435e13,8.4e13,7.07e13,2.074e13,0.579871,0.608062,5.65232,0.0223,1620569405,1.02956,13.8607,8.4e13,13.9243,1.3104e13
6,1619882298,-1,46.9333,134.869,85.2907,-32.87,0.42,29.38,664.42,737.05,7.239e13,6.482e13,6.325e13,6.602e13,6.553e13,2.278e13,0.734503,0.734503,6.13975,0.06525,1619882298,1.02875,13.8597,7.239e13,13.8597,1.12928e13
7,1620570495,-1,10.0025,186.166,125.959,173.74,249.63,270.22,650.17,679.69,6.975e13,6.799e13,6.33e13,6.143e13,5.133e13,3.735e13,0.529195,0.529195,6.26543,0.04754,1620570495,1.01609,13.8435,6.975e13,13.8435,1.0881e13
8,1621113337,-1,39.2183,196.085,87.0058,-92.91,99.81,237.01,560.58,643.33,5.08e13,5.55e13,5.615e13,6.024e13,3.472e13,1.32e13,0.503591,0.516679,4.35401,0.08217,1621113337,0.914195,13.7059,6.024e13,13.7799,9.39744e12
9,1620494893,-1,38.9772,181.184,67.201,-119.43,12.79,11.14,568.63,639.93,4.591e13,4.567e13,4.37e13,4.085e13,3.21e13,1.713e13,0.473207,0.475291,6.38986,0.03874,1620494893,0.883866,13.6619,4.591e13,13.6619,7.16196e12
10,1620494673,-1,28.9453,165.548,73.521,28.3,-4.23,-105.61,586.93,628.04,4.428e13,4.125e13,3.994e13,3.841e13,3.449e13,2.469e13,0.381623,0.381623,7.59335,0.03238,1620494673,0.87328,13.6462,4.428e13,13.6462,6.90768e12


In [31]:
# make a merged dataframe of galaxies and halos
mockdf = DataFrame(delta = deltas, halo_id = mockgals[:, 1], subvol = mockgals[:, 6])

# keep only the central galaxies
mockdf = mockdf[mockgals[:, 5] .== 1, :]

# merge with the halos on halo_id
mockdf = join(mockdf, halodf, on = :halo_id)

,delta,halo_id,subvol,halo_upid,x,y,z,vx,vy,vz,hfoo1,hfoo2,halo_mvir,halo_mz_05,halo_mz_1,halo_mz_2,halo_mz_4,halo_mz_8,halo_ac,halo_ac_mpeak,halo_nfw_conc,halo_spin,hfoo3,halo_rvir,logMh,Mpeak,logMpeak,Mbar
1,2.66376,1.62049e9,4.0,-1,23.8908,178.352,71.8972,254.19,-195.17,-253.54,758.46,780.72,1.303e14,1.117e14,1.016e14,9.289e13,6.113e13,4.885e13,0.51462,0.51462,4.34886,0.03889,1620490488,1.25141,14.1149,1.303e14,14.1149,2.03268e13
2,2.82305,1.62049e9,4.0,-1,54.2385,147.212,83.8446,132.18,-243.42,241.66,746.88,787.27,1.031e14,1.009e14,9.834e13,8.531e13,6.773e13,5.168e13,0.602988,0.602988,6.31183,0.03163,1620489517,1.15745,14.0133,1.031e14,14.0133,1.60836e13
3,0.274351,1.62057e9,4.0,-1,17.5035,142.147,121.233,206.25,52.6,75.76,800.83,833.51,1.023e14,9.305e13,8.565e13,8.209e13,7.753e13,5.354e13,0.465599,0.465599,8.92032,0.01292,1620568787,1.15445,14.0099,1.023e14,14.0099,1.59588e13
4,0.194704,1.62057e9,4.0,-1,30.9299,185.085,116.149,100.4,211.18,314.45,759.85,808.61,8.734e13,8.65e13,8.481e13,7.882e13,6.102e13,4.87e13,0.474902,0.474902,8.7816,0.02291,1620569234,1.09519,13.9412,8.734e13,13.9412,1.3625e13
5,0.433645,1.62057e9,4.0,-1,3.68962,147.032,119.642,318.78,-36.68,214.57,655.71,762.05,7.256e13,7.268e13,7.435e13,8.4e13,7.07e13,2.074e13,0.579871,0.608062,5.65232,0.0223,1620569405,1.02956,13.8607,8.4e13,13.9243,1.3104e13
6,1.86729,1.61988e9,4.0,-1,46.9333,134.869,85.2907,-32.87,0.42,29.38,664.42,737.05,7.239e13,6.482e13,6.325e13,6.602e13,6.553e13,2.278e13,0.734503,0.734503,6.13975,0.06525,1619882298,1.02875,13.8597,7.239e13,13.8597,1.12928e13
7,0.274351,1.62057e9,4.0,-1,10.0025,186.166,125.959,173.74,249.63,270.22,650.17,679.69,6.975e13,6.799e13,6.33e13,6.143e13,5.133e13,3.735e13,0.529195,0.529195,6.26543,0.04754,1620570495,1.01609,13.8435,6.975e13,13.8435,1.0881e13
8,1.46906,1.62111e9,4.0,-1,39.2183,196.085,87.0058,-92.91,99.81,237.01,560.58,643.33,5.08e13,5.55e13,5.615e13,6.024e13,3.472e13,1.32e13,0.503591,0.516679,4.35401,0.08217,1621113337,0.914195,13.7059,6.024e13,13.7799,9.39744e12
9,0.274351,1.62049e9,4.0,-1,38.9772,181.184,67.201,-119.43,12.79,11.14,568.63,639.93,4.591e13,4.567e13,4.37e13,4.085e13,3.21e13,1.713e13,0.473207,0.475291,6.38986,0.03874,1620494893,0.883866,13.6619,4.591e13,13.6619,7.16196e12
10,0.911527,1.62049e9,4.0,-1,28.9453,165.548,73.521,28.3,-4.23,-105.61,586.93,628.04,4.428e13,4.125e13,3.994e13,3.841e13,3.449e13,2.469e13,0.381623,0.381623,7.59335,0.03238,1620494673,0.87328,13.6462,4.428e13,13.6462,6.90768e12


In [32]:
# now to add galaxy stellar masses according to the best fit stellar mass halo mass relation
ths = [18, 19, 20]

# read in the parameters
th = ths[sample_num]
params = squeeze(readdlm("../dat/optim/M$(@sprintf("%d", th))_mass_pars.dat"), 2)

# add stellar mass to dataframe
mockdf[:stelM] = gen_mstars(Array(mockdf[:Mpeak]), Array(mockdf[:Mbar]), params, fix_sig = false)

# and add log stellar mass
mockdf[:log10M] = log10.(mockdf[:stelM])

311248-element Array{Float64,1}:
 10.1213 
 10.1378 
  9.58766
  9.27448
  9.69037
 10.0152 
  9.59987
  9.77239
 10.2787 
 10.5755 
 10.1182 
  9.5999 
  9.16873
  ⋮      
  9.15838
  8.85897
  9.27235
  9.39619
  8.83858
  8.30013
  8.87814
  8.67135
  9.25434
  8.80469
  8.4353 
  8.69023

In [33]:
# set the mass ranges to restrict to for each sample
massmins = [9.4, 9.8, 10.3]
massmaxs = [9.8, 10.3, 11.0]

# keep only the centrals within the massrange of interest for the modeling
mockdf = mockdf[massmins[sample_num] .<= mockdf[:log10M] .< massmaxs[sample_num], :]
datdf = datdf[massmins[sample_num] .<= datdf[:log10M] .< massmaxs[sample_num], :]

,foo,galid,groupid,cenid,rband,Psat,Mh,foo2,foo3,foo4,projR,projrad,angRh,ρ_env,foo5,M_r,M_g,cz,Dn4000,H_delta,logsSFR,stelM,ra,dec,vdisp,S2N,sersic,conc,KplusA,R_exp,surfdens1kpc,surfdensR_e,surfdensR_eo2,vdisp_err,Bulge2Tlr,B2T_err,GMoR_e,R_e,logMh,Dₙ4000,logRe,logsurf,logv,delta,log10M,logρ
1,PROB10,3,26747,3,-20.9869,0.0,3.03811e12,0.0,3.03811e12,0.0,0.0,0.0,0.00233309,0.7093,GALDAT_CORR,-20.9869,-20.209,16203.8,1.72142,-0.90094,-10.8185,3.25081e10,0.669571,0.003677,137.339,26.5502,1.69591,2.52066,0,11.5708,2.94449e8,6.47466e8,1.5229e9,5.46311,0.32,0.01,85.6401,0.005875,12.4826,1.72142,-2.23099,9.18267,2.13779,-0.2907,10.512,-0.14917
2,PROB10,17,46451,17,-20.4593,0.0,1.67686e12,0.0,1.67686e12,0.0,0.0,0.0,0.0025809,1.419,GALDAT_CORR,-20.4593,-19.5225,11982.8,1.93181,-2.23705,-11.9309,2.32574e10,0.977202,0.007641,224.655,43.0824,6.0,3.28954,0,4.18451,8.5071e8,2.04648e9,2.46656e9,4.68927,0.71,0.01,107.792,0.002568,12.2245,1.93181,-2.5904,9.39209,2.35152,0.419,10.3666,0.151982
3,PROB10,21,29108,21,-21.3315,0.0,2.77697e12,0.0,2.77697e12,0.0,0.0,0.0,0.00300727,1.347,GALDAT_CORR,-21.3315,-20.6573,12168.4,1.54253,0.02676,-10.6549,3.0963e10,0.976836,0.010722,114.26,24.3395,1.22282,1.99674,0,13.8139,9.59716e7,1.81312e8,6.10891e8,6.63475,0.12,0.0,68.4435,0.009579,12.4436,1.54253,-2.01868,8.78596,2.05789,0.347,10.4908,0.129368
4,PROB10,53,15187,53,-21.1077,0.0,5.48889e12,0.0,5.48889e12,0.0,0.0,0.0,0.00212182,1.225,GALDAT_CORR,-21.1077,-20.1396,21779.8,1.99379,-2.14072,-12.3104,4.47538e10,0.670143,0.016795,207.608,29.2532,5.42434,3.27889,0,8.3279,1.70497e8,4.23904e8,1.39671e9,6.47959,0.59,0.01,93.2896,0.009548,12.7395,1.99379,-2.02009,9.14511,2.31724,0.225,10.6508,0.0881361
5,PROB10,81,36849,81,-20.5768,0.0,2.15532e12,0.0,2.15532e12,0.0,0.0,0.0,0.00150435,2.096,GALDAT_CORR,-20.5768,-19.6817,22505.9,1.81657,0.78196,-11.7147,2.6923e10,0.721144,0.017946,131.568,23.2803,2.96082,2.80572,0,5.07165,6.07125e8,1.14361e9,1.36953e9,7.34734,0.66,0.01,91.2233,0.00343,12.3335,1.81657,-2.46471,9.13657,2.11915,1.096,10.4301,0.321391
6,PROB10,83,24536,83,-20.8211,0.0,3.32973e12,0.0,3.32973e12,0.0,0.0,0.0,0.00530651,4.456,GALDAT_CORR,-20.8211,-19.8558,7303.35,1.81796,-0.95315,-11.2977,3.41598e10,0.721953,0.015959,195.688,40.283,2.68589,3.18527,0,6.62674,3.83006e8,9.00621e8,1.91893e9,4.62724,-999.0,0.0,96.8497,0.004839,12.5224,1.81796,-2.31524,9.28306,2.29156,3.456,10.5335,0.648945
7,PROB10,85,52256,85,-20.6159,0.0,1.47419e12,0.0,1.47419e12,0.0,0.0,0.0,0.00117909,0.1147,GALDAT_CORR,-20.6159,-19.8719,25347.1,1.46414,1.36506,-10.4219,2.15102e10,0.722218,0.017504,104.551,13.9363,1.00097,2.23405,0,11.855,2.11281e8,3.88927e8,5.63207e8,10.3245,0.07,0.02,64.1593,0.005995,12.1686,1.46414,-2.22221,8.75067,2.01933,-0.8853,10.3326,-0.940437
8,PROB10,90,25369,90,-20.8662,0.0,3.21402e12,0.0,3.21402e12,0.0,0.0,0.0,0.00128072,0.1207,GALDAT_CORR,-20.8662,-20.0167,30360.1,1.67551,0.25085,-11.2725,3.35314e10,0.728526,0.017561,164.879,22.5892,1.50872,2.50878,0,5.63503,8.75448e8,1.44936e9,1.55069e9,7.87287,0.39,0.02,94.8622,0.003274,12.507,1.67551,-2.48492,9.19052,2.21717,-0.8793,10.5255,-0.918293
9,PROB10,93,10451,93,-21.3781,0.0,7.98381e12,0.0,7.98381e12,0.0,0.0,0.0,0.00619652,2.37,GALDAT_CORR,-21.3781,-20.4015,8376.85,1.85702,-1.32328,-12.1329,5.50736e10,0.736818,0.017271,202.039,42.1878,3.71331,2.85523,0,7.29463,4.53193e8,1.0304e9,2.5889e9,4.49715,-999.0,0.0,118.282,0.005232,12.9022,1.85702,-2.28133,9.41312,2.30544,1.37,10.7409,0.374748
10,PROB10,112,1095,112,-21.8533,0.0,4.79077e13,1.0,4.79077e13,1.0,0.0,0.0,0.00469486,0.9654,GALDAT_CORR,-21.8533,-21.0492,20246.1,1.86562,-2.37048,-11.8447,8.58297e10,0.760278,0.017962,181.873,27.0336,3.35338,2.01356,0,16.9032,1.31262e8,2.7979e8,1.27374e9,6.26233,0.13,0.0,92.3567,0.012462,13.6804,1.86562,-1.90441,9.10508,2.25977,-0.0346,10.9336,-0.0152927


In [34]:
# shift the environment densities to match those of the data, in case of difference
logdeltas = log10.(mockdf[:delta] + 1)
deltamean = mean(logdeltas)
deltavar = var(logdeltas)
println("       ", "data", "    |    ", "mock")
println("means: ", mean(datdf[:logρ]), " | ", deltamean)
println("devs:  ", sqrt(var(datdf[:logρ])), " | ", sqrt(deltavar))
renormed = (logdeltas - deltamean) / sqrt(deltavar)
mockdf[:delta] = sqrt(var(datdf[:logρ])) * renormed + mean(datdf[:logρ])

       data    |    mock
means: 0.13237964711138933 | 0.1846097498643532
devs:  0.36766927055337684 | 0.31137488085175424


101807-element Array{Float64,1}:
  0.246646 
  0.402123 
 -0.0878743
  0.287693 
 -0.130633 
  0.0846726
  0.0846726
 -0.029758 
  0.287693 
  0.126845 
 -0.0117628
  0.61788  
 -0.177285 
  ⋮        
  0.901144 
  0.527482 
  0.533623 
  0.886016 
 -1.02764  
  0.612674 
  0.508603 
  0.297461 
  0.126845 
 -0.256335 
  0.652915 
  0.224821 

In [36]:
# now to get the mock numbers to match the quenched fraction in the data
nqbins = [10, 20, 20]  # number of mass bins to use for the quenched fraction

# get the massbins for computing quenched fraction
quenmasses = linspace(massmins[sample_num], massmaxs[sample_num], nqbins[sample_num] + 1)

# compute fractions for the data
qfracs = zeros(nqbins[sample_num])
for j in 1:nqbins[sample_num]
    ntot = size(datdf[quenmasses[j] .<= datdf[:log10M] .< quenmasses[j + 1], :])[1]
    nquens = size(quendf[quenmasses[j] .<= quendf[:log10M] .< quenmasses[j + 1], :])[1]
    qfracs[j] = nquens / ntot
    println(ntot)
end
println(qfracs)
    
mockdf = get_quenfrac(mockdf, quenmasses, qfracs)

5164
4979
4704
4233
4049
3622
3256
3001
2660
2397
1934
1618
1371
1168
980
837
625
505
388
293
[0.660922, 0.6905, 0.718537, 0.737302, 0.767597, 0.784373, 0.816339, 0.852716, 0.876316, 0.876095, 0.91727, 0.932633, 0.92779, 0.932363, 0.960204, 0.965352, 0.9648, 0.972277, 0.987113, 0.982935]


,delta,halo_id,subvol,halo_upid,x,y,z,vx,vy,vz,hfoo1,hfoo2,halo_mvir,halo_mz_05,halo_mz_1,halo_mz_2,halo_mz_4,halo_mz_8,halo_ac,halo_ac_mpeak,halo_nfw_conc,halo_spin,hfoo3,halo_rvir,logMh,Mpeak,logMpeak,Mbar,stelM,log10M
1,-0.130633,1.63526e9,23.0,-1,335.486,194.784,158.714,-100.61,38.81,292.87,179.75,207.71,1.433e12,1.449e12,1.435e12,1.48e12,1.233e12,9.102e11,0.613141,0.658105,5.87298,0.03959,1635259433,0.278291,12.1562,1.48e12,12.1703,2.3088e11,2.01631e10,10.3046
2,0.126845,1.62664e9,7.0,-1,120.988,293.416,92.5466,449.74,440.02,-114.95,174.72,194.3,1.519e12,1.407e12,1.312e12,1.267e12,1.13e12,5.086e11,0.416704,0.416704,4.82677,0.13775,1626639600,0.283751,12.1816,1.519e12,12.1816,2.36964e11,2.08578e10,10.3193
3,0.912934,1.62574e9,15.0,-1,147.633,174.91,296.593,-1157.53,-830.79,-216.52,392.0,441.45,1.163e13,1.131e13,1.129e13,1.112e13,1.05e13,5.202e12,0.448062,0.448062,9.65841,0.12678,1625739164,0.559256,13.0656,1.163e13,13.0656,1.81428e12,2.11536e10,10.3254
4,0.190236,1.62066e9,5.0,-1,96.3887,153.813,185.629,-483.72,-242.26,263.99,265.43,301.39,4.94e12,4.561e12,3.504e12,3.257e12,3.256e12,2.435e12,0.391677,0.391677,5.51031,0.22861,1620655543,0.420397,12.6937,4.94e12,12.6937,7.7064e11,2.01179e10,10.3036
5,-0.108808,1.63367e9,21.0,-1,307.776,43.6341,315.343,0.83,-330.96,-6.17,177.34,181.52,9.723e11,8.629e11,6.778e11,6.529e11,6.115e11,5.547e11,0.29022,0.29022,12.0488,0.27182,1633672150,0.244541,11.9878,9.723e11,11.9878,1.51679e11,2.03366e10,10.3083
6,0.11317,1.62173e9,4.0,-1,69.0045,260.789,83.8874,-233.67,-5.51,-12.88,328.24,338.99,6.171e12,6.031e12,5.841e12,5.649e12,5.285e12,3.794e12,0.290548,0.290548,10.4297,0.06836,1621726525,0.452761,12.7904,6.171e12,12.7904,9.62676e11,2.14427e10,10.3313
7,0.694705,1.63148e9,16.0,-1,245.737,311.761,43.1658,-526.13,513.7,-690.99,219.4,236.76,1.766e12,1.741e12,1.701e12,1.655e12,1.518e12,1.274e12,0.325684,0.325684,12.1335,0.07079,1631484915,0.298365,12.247,1.766e12,12.247,2.75496e11,2.13755e10,10.3299
8,0.514974,1.63319e9,27.0,-1,298.32,399.108,382.285,487.49,432.01,86.63,462.49,476.96,1.565e13,1.506e13,1.473e13,1.42e13,1.284e13,1.12e13,0.325502,0.325502,12.6759,0.04847,1633194058,0.617432,13.1945,1.565e13,13.1945,2.4414e12,2.02089e10,10.3055
9,0.685709,1.61899e9,2.0,-1,70.9309,15.578,257.796,83.91,324.42,-49.61,166.86,174.45,7.689e11,7.594e11,7.434e11,7.139e11,6.163e11,5.607e11,0.329726,0.329726,13.5006,0.10898,1618994141,0.226138,11.8859,7.689e11,11.8859,1.19948e11,2.1313e10,10.3286
10,0.61788,1.61974e9,3.0,-1,55.3104,72.7393,361.84,309.83,-274.46,-16.65,299.89,361.22,7.136e12,6.798e12,6.501e12,3.563e12,2.733e12,2.48e12,0.486893,0.486893,4.73974,0.10203,1619741475,0.475228,12.8535,7.136e12,12.8535,1.11322e12,2.13229e10,10.3288


In [37]:
# finally leave only the relevant columns in the mock for modeling and save to disk
mockdf = mockdf[:, [:halo_id, :halo_mvir, :halo_rvir, :halo_nfw_conc, :halo_spin, :halo_mz_05, :halo_mz_1,
                    :logMh, :logMpeak, :delta, :stelM, :log10M, :subvol]]
outfile = "../dat/mocks/M$(@sprintf("%d", th))_cenquen_mock.csv"
CSV.write(outfile, mockdf)

CSV.Sink{Void,DataType}(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        missingstring: ""
        dateformat: nothing
        decimal: '.'
        truestring: 'true'
        falsestring: 'false'
        internstrings: true, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "../dat/mocks/M20_cenquen_mock.csv", 114, true, String["halo_id", "halo_mvir", "halo_rvir", "halo_nfw_conc", "halo_spin", "halo_mz_05", "halo_mz_1", "logMh", "logMpeak", "delta", "stelM", "log10M", "subvol"], 13, false, Val{false})